<a href="https://colab.research.google.com/github/jung0228/ML/blob/main/%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***선행작업***

In [2]:
!pip install konlpy
!pip install collections
!pip install wordcloud
!pip install sweetviz


     |████████████████████████████████| 19.4 MB 446 kB/s 
     |████████████████████████████████| 448 kB 47.2 MB/s 
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
     |████████████████████████████████| 15.1 MB 4.5 MB/s 


In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
import sweetviz as sv
import missingno as msno
# 다운받은 csv를 pandas의 DataFrame 형식으로 불러옵니다.
data = pd.read_csv('train.csv')

data[20:100] # 데이터를 확인합니다.

,id,document,label
20,21,"처절한 영화, 슬픈 영화, 씁쓸-한 영화...",1
21,22,처음부터 끝까지 유치 하기 그지 없다....난해한 영화,0
22,23,전쟁? 아이들의 목소리가 참 거슬린다.,0
23,24,멋진 상상력! 아름다운 영화. 여배우 소이의 재발견!,1
24,25,그 시대엔 이 영화가 최고였음,1
...,...,...,...
95,96,표 값과 시간이 아까웠던 영화,0
96,97,쩌는 시나리오 쩌는 연기 환상의 소피아 결론은 제이미 벨 이 축복받은 놈,1
97,98,너무 기분좋게봣습니다~ 적극 추천!!,1
98,99,왜 그랬니? 피트 출현할 영화가 그렇게 없디?,0


# ***데이터 분석하기***

## ***그냥 둘러보기***

In [5]:
df_train = data.copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [6]:
#결측값은 없는지 확인하기
for col in df_train.columns:
    msg = 'column: {:>20} \t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

column:                   id 	 Percent of NaN value: 0.00%
column:             document 	 Percent of NaN value: 0.00%
column:                label 	 Percent of NaN value: 0.00%


## ***(문장)끼리 비교해보기***

In [7]:
val_count = data['label'].value_counts() # 유니크값의 개수를 확인합니다.
print('라벨 0인 리뷰 개수:', val_count[0])
print('라벨 1인 리뷰 개수:', val_count[1])

라벨 0인 리뷰 개수: 2564
라벨 1인 리뷰 개수: 2436


In [8]:
len_1 = 0
len_0 = 0
for i in range(len(df_train)):
  str = df_train['document'][i]
  if df_train['label'][i] == 1:
    len_1 += len(str)
  if df_train['label'][i] == 0:
    len_0 += len(str)
  
print(f"긍정 평가 평균 길이 : {len_1/val_count[1]}\n부정 평가 평균 길이 : {len_0/val_count[0]}")

긍정 평가 평균 길이 : 27.188013136288998
부정 평가 평균 길이 : 27.085023400936038


##***(단어)의 통계적인 비율 살펴보기***

In [13]:
bad_word = {}
bad_num = 0

for i in range(5000):
  if df_train['label'][i] == 0:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in bad_word:
        bad_word[i] = 1
        bad_num += 1
      else:
        bad_word[i] += 1
        bad_num += 1

# lambda x : x[1] 
d1 = sorted(bad_word.items(), key=lambda x: x[1], reverse=True) 
#print(d1)
for i in range(20):
  print(d1[i])
print(f"전체 악평 단어 개수 : {bad_num}")


('영화', 754)
('이', 143)
('진짜', 133)
('점', 130)
('평점', 119)
('왜', 116)
('재미', 105)
('정말', 104)
('스토리', 102)
('최악', 98)
('시간', 98)
('내용', 90)
('것', 89)
('뭐', 87)
('감독', 79)
('거', 78)
('내', 77)
('그냥', 77)
('돈', 75)
('연기', 73)
전체 악평 단어 개수 : 11854


In [23]:
good_word = {}
good_num = 0

for i in range(5000):
  if df_train['label'][i] == 1:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in good_word:
        good_word[i] = 1
        good_num += 1
      else:
        good_word[i] += 1
        good_num += 1

# lambda x : x[1] 
d2 = sorted(good_word.items(), key=lambda x: x[1], reverse=True) 
for i in range(20):
  print(d2[i])
print(f"전체 호평 단어 개수 : {good_num}")

('영화', 769)
('정말', 216)
('최고', 214)
('진짜', 136)
('감동', 135)
('연기', 111)
('드라마', 90)
('보고', 89)
('때', 83)
('이', 79)
('사랑', 74)
('다시', 72)
('평점', 67)
('점', 67)
('것', 66)
('생각', 65)
('말', 63)
('명작', 61)
('내', 58)
('배우', 58)
전체 호평 단어 개수 : 10995


In [15]:
print(f"부정 단어 가짓수 : {len(d1)}")
print(f"긍정 단어 가짓수 : {len(d2)}")

부정 단어 가짓수 : 3227
긍정 단어 가짓수 : 3110


In [ ]:
def persent(n, word_num, d):
  sum = 0
  for i in range(n):
    sum += d[i][1]/word_num
  print(f"단어 {n}개 사용했을 때 비율 : {round(sum*100, 2)} %")

In [ ]:
print("부정 단어")
for i in range(500, 2000, 200):
  persent(i, bad_num, d1)
print("\n긍정단어")
for i in range(500, 2000, 200):
  persent(i, good_num, d2)

부정 단어
단어 500개 사용했을 때 비율 : 68.23 %
단어 700개 사용했을 때 비율 : 73.59 %
단어 900개 사용했을 때 비율 : 77.42 %
단어 1100개 사용했을 때 비율 : 80.79 %
단어 1300개 사용했을 때 비율 : 83.74 %
단어 1500개 사용했을 때 비율 : 85.43 %
단어 1700개 사용했을 때 비율 : 87.12 %
단어 1900개 사용했을 때 비율 : 88.81 %

긍정단어
단어 500개 사용했을 때 비율 : 68.85 %
단어 700개 사용했을 때 비율 : 74.08 %
단어 900개 사용했을 때 비율 : 77.72 %
단어 1100개 사용했을 때 비율 : 81.36 %
단어 1300개 사용했을 때 비율 : 83.54 %
단어 1500개 사용했을 때 비율 : 85.36 %
단어 1700개 사용했을 때 비율 : 87.18 %
단어 1900개 사용했을 때 비율 : 88.99 %


In [ ]:
#겹치는 단어는 얼마나 있을까?
for i in range(1000):
  for j in range(1000):


### ***단어 목록 살펴보기 (전체 다 보면 수위가 높은 단어들도 많이보인다)***

In [20]:
for i in range(40):
  print(d1[i][0])

영화
이
진짜
점
평점
왜
재미
정말
스토리
최악
시간
내용
것
뭐
감독
거
내
그냥
돈
연기
쓰레기
배우
좀
말
이건
걸
끝
별로
보고
더
액션
나
그
하나
사람
생각
완전
정도
개
드라마


In [22]:
for i in range(40):
  print(d2[i][0])

영화
정말
최고
진짜
감동
연기
드라마
보고
때
이
사랑
다시
평점
점
것
생각
말
명작
내
배우
수
더
사람
왜
완전
마지막
가슴
볼
이야기
그
작품
재미
또
꼭
중
역시
거
인생
나
지금


# ***예측해보기***
